In [1]:
from arctic import Arctic
from arctic.date import DateRange
from arctic.exceptions import NoDataFoundException
from arctic.date import DateRange
import pandas as pd
from ib_insync import IB, ContFuture, util, Future, Forex, Contract
from typing import Union
import pickle
from datetime import datetime, timedelta
import sys
sys.path.append('/home/tomek/ib_tools')
from datastore import ArcticStore

/home/tomek/.virtualenvs/ib/lib/python3.7/site-packages/arctic/store/_pandas_ndarray_store.py:6: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import DataFrame, Series, Panel


In [2]:
util.startLoop()
ib = IB()
ib.connect(port=4002, clientId=9)

<IB connected to 127.0.0.1:4002 clientId=9>

In [3]:
ib.isConnected()

True

In [ ]:
Arctic('localhost').list_libraries()

In [4]:
store = ArcticStore('TRADES_30_secs')

Library created, but couldn't enable sharding: no such command: 'enablesharding'. This is OK if you're not 'admin'


In [ ]:
keys = store.keys()

In [ ]:
missing = []
for key in keys:
    if store.read_metadata(key)['object'] is None:
        missing.append(key)
missing

In [ ]:
store.read_metadata(missing[0])

In [ ]:
new_missing = []
for key in keys:
    if store.read_metadata(key)['object'] is None:
        new_missing.append(key)
new_missing

In [ ]:
objects = []
for key in missing:
    meta = store.read_metadata(key)
    fields = {}
    for field in ['symbol', 'tradingClass', 'exchange', 'lastTradeDateOrContractMonth']:
        fields[field] = meta.get(field)
    objects.append(Future(**fields, includeExpired=True))
objects

In [ ]:
ib.qualifyContracts(*objects)

In [ ]:
contfutures = []
for object in objects:
    contfutures.append(ContFuture(**object.nonDefaults()).update(secType='CONTFUT'))
contfutures

In [ ]:
import pickle
for object in contfutures:
    store.write_metadata(object, {'object': pickle.dumps(object)})

In [ ]:
objects

In [5]:
for key in store.keys():
    store.delete_metadata_item(key, 'expiry')

In [6]:
store.keys()

['6EM0_CONTFUT',
 '6EU0_CONTFUT',
 '6JM0_CONTFUT',
 '6JU0_CONTFUT',
 'BZZ0_CONTFUT',
 'CLN0_CONTFUT',
 'CLU0_CONTFUT',
 'ESM0_CONTFUT',
 'ESU0_CONTFUT',
 'FCEM0_CONTFUT',
 'FDAX_JUN_20_CONTFUT',
 'FDAX_SEP_20_CONTFUT',
 'GCQ0_CONTFUT',
 'GEM0_CONTFUT',
 'GEU0_CONTFUT',
 'M2KM0_CONTFUT',
 'M2KU0_CONTFUT',
 'M6EM0_CONTFUT',
 'M6EU0_CONTFUT',
 'MESM0_CONTFUT',
 'MESU0_CONTFUT',
 'MFCN0_CONTFUT',
 'MGCQ0_CONTFUT',
 'MJYM0_CONTFUT',
 'MJYU0_CONTFUT',
 'MNQM0_CONTFUT',
 'MNQU0_CONTFUT',
 'MYM_SEP_20_CONTFUT',
 'NKDM0_CONTFUT',
 'NKDU0_CONTFUT',
 'NQM0_CONTFUT',
 'NQU0_CONTFUT',
 'RTYM0_CONTFUT',
 'RTYU0_CONTFUT',
 'YM_JUN_20_CONTFUT',
 'YM_SEP_20_CONTFUT',
 'ZB_JUN_20_CONTFUT',
 'ZB_SEP_20_CONTFUT',
 'ZF_JUN_20_CONTFUT',
 'ZF_SEP_20_CONTFUT',
 'ZN_JUN_20_CONTFUT',
 'ZN_SEP_20_CONTFUT',
 'ZS_JUL_20_CONTFUT']

In [7]:
store.read_metadata('ESM0_CONTFUT')

{'secType': 'CONTFUT',
 'conId': 357283364,
 'symbol': 'ES',
 'lastTradeDateOrContractMonth': '20200619',
 'multiplier': '50',
 'exchange': 'GLOBEX',
 'currency': 'USD',
 'localSymbol': 'ESM0',
 'tradingClass': 'ES',
 'repr': "ContFuture(conId=357283364, symbol='ES', lastTradeDateOrContractMonth='20200619', multiplier='50', exchange='GLOBEX', currency='USD', localSymbol='ESM0', tradingClass='ES')",
 'object': b'\x80\x03cib_insync.contract\nContFuture\nq\x00)\x81q\x01N}q\x02(X\x07\x00\x00\x00secTypeq\x03X\x07\x00\x00\x00CONTFUTq\x04X\x05\x00\x00\x00conIdq\x05J$\xb6K\x15X\x06\x00\x00\x00symbolq\x06X\x02\x00\x00\x00ESq\x07X\x1c\x00\x00\x00lastTradeDateOrContractMonthq\x08X\x08\x00\x00\x0020200619q\tX\x06\x00\x00\x00strikeq\nG\x00\x00\x00\x00\x00\x00\x00\x00X\x05\x00\x00\x00rightq\x0bX\x00\x00\x00\x00q\x0cX\n\x00\x00\x00multiplierq\rX\x02\x00\x00\x0050q\x0eX\x08\x00\x00\x00exchangeq\x0fX\x06\x00\x00\x00GLOBEXq\x10X\x0f\x00\x00\x00primaryExchangeq\x11h\x0cX\x08\x00\x00\x00currencyq\x12X\x03

In [ ]:
store.read('ESM0_CONTFUT', start_date='20180601')

In [ ]:
nq = ContFuture('NQ', exchange='globex')
ib.qualifyContracts(nq)

In [ ]:
store.read('NQM0_CONTFUT', start_date='20180601')

In [ ]:
store.read('ESM0_CONTFUT')

In [ ]:
store.latest_contfutures

In [ ]:
store.review('secType', 'exchange')

In [ ]:
store.keys()

In [ ]:
store.read(nq, '20180605')

In [ ]:
from datastore import Store

In [ ]:
old_store = Store()

In [ ]:
old_store.keys()

In [ ]:
old_store.read('/cont/min/NQ_20200320_GLOBEX_USD').sort_index(ascending=True).loc['20180701':].head(10)

In [ ]:
store.read(nq, '20180701', end).head(10)

In [ ]:
date_range = DateRange('2018-05-19 00:00:00', '2019-12-31 00:00:00')

In [ ]:
start = '2018-05-19 00:00:00'
end = '2019-12-31 00:00:00'

In [ ]:
nq = ContFuture(conId=371749745, symbol='NQ', lastTradeDateOrContractMonth='20200918', multiplier='20', exchange='GLOBEX', currency='USD', localSymbol='NQU0', tradingClass='NQ')


In [ ]:
nq

In [ ]:
bars = ib.reqHistoricalData(
            nq,
            endDateTime= '20180602 00:00:00',
            durationStr='3 D',
            barSizeSetting='30 secs',
            whatToShow='TRADES',
            useRTH=False,
            formatDate=1)

In [ ]:
df = util.df(bars)
df

In [ ]:
store.read(nq, '20180701', end)

In [ ]:
l = store._contfutures_dict()
l

In [ ]:
x = {}
index = -2
for c, d in l.items():
    keys_list = list(d.keys())
    if len(keys_list) + index < 0:
        index = 0
    x[c] = d[keys_list[index]]
x

In [ ]:
list(d.keys())[0]

In [ ]:
store.latest_contfutures(-2)

In [ ]:
store.contfuture_contract_object('NQ', -2)

In [ ]:
df = store.read('ESM0_CONTFUT', '20180601', '20191231')
df

In [ ]:
(df['close'][-1] - df['open'][0])*20

In [ ]:
store.read_metadata('NQM0_CONTFUT')

In [ ]:
from collections import defaultdict
c = defaultdict(dict)
for k, v in d.items():
    for i in sorted(v):
        c[k].update({i: v[i]})  
c

In [ ]:
d

In [ ]:
c

In [ ]:
store.read(nq)

In [ ]:
%%timeit
store.date_range()

In [ ]:
store.read_object('NQU0_CONTFUT')

In [ ]:
store.read_metadata('NQU0_CONTFUT')

In [ ]:
store.read_object('YM_JUN_20_CONTFUT')

In [ ]:
store.latest_contfutures

In [ ]:
store.contfuture('NQ')

In [ ]:
def hist(days_at_a_time):
    dt = ''
    barsList = []
    i = 10/days_at_a_time
    #i=1
    while i>0:
        bars = ib.reqHistoricalData(
            nq,
            endDateTime=dt,
            durationStr=str(days_at_a_time) + ' D',
            barSizeSetting='1 min',
            whatToShow='TRADES',
            useRTH=False,
            formatDate=1)
        #lib.append('edek', bars)
        if not bars:
            break
            dt -= timedelta(days=1)
            print(f'new dt: {dt}')
            continue
        barsList.append(bars)
        dt = bars[0].date
        print(dt)
        i -= 1
    allBars = [b for bars in reversed(barsList) for b in bars]
    df = util.df(allBars)
    return df

In [ ]:
bars = ib.reqHistoricalData(
            mes,
            endDateTime= '20190506 00:00:00',
            durationStr='1 D',
            barSizeSetting='30 secs',
            whatToShow='TRADES',
            useRTH=False,
            formatDate=1)

In [ ]:
nq

In [ ]:
nq.loc['20190101':]